# Analysis_201
This notebook deals with the analysis of my second research question 'A summary of  visualisations and packages use.'

## Purpose 
* This notebook will read in larger dataset from Prep_200 containing information on packages and headings
* What will be looked at:
    * Packages used by users
    * Headings used

In [1]:
#importing relevant libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

In [2]:
df_cell_new = pd.read_csv('../data/CSV_files/new_cell_info.csv') # reading in csv 

In [3]:
df_cell_new.head()

,nb_id,nb_language,worksheet_index,cell_index,num_imports,imports,num_headings,headings,header_level
0,400000,julia,0,NaN,1,"['DataFrames,']",0,[],[]
1,400000,julia,1,NaN,0,[],0,[],[]
2,400000,julia,2,NaN,0,[],0,[],[]
3,400000,julia,3,NaN,0,[],0,[],[]
4,400000,julia,4,NaN,0,[],0,[],[]


## Packages used

In [4]:
df_cell_new['imports'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in df_cell_new['imports']])

Removing of square brackets

In [5]:
df_imports = df_cell_new.replace('', np.nan) # replacing space with NaN values

In [6]:
df_imports = df_imports.dropna(subset=['imports']) #droping rows with no imports

In [7]:
df_imports['num_imports'].mean()

2.79429072205574

There is an average of 2.7 (3) imports per cell when nan values are dropped. 

In [8]:
df_imports.groupby('nb_id')['num_imports'].sum().mean()

7.48166549869686

There is an average of 7.5 imports per notebook.

In [9]:
imports_new = pd.Series(df_imports.imports.str.split(', ').sum()).value_counts() 
imports_new.head(10)

numpy                109356
matplotlib.pyplot     68605
pandas                62087
sklearn               22173
os                    20754
matplotlib            19133
sklearn.metrics       18251
math                  17349
seaborn               16452
IPython.display       15921
dtype: int64

we can see numpy is the most used library with 109,356 occurrences thus appearing in over 50\% of the notebooks. The second most used library is matplotlib and the third pandas, indicating a lot of users are creating visualizations in their notebooks. 

In [10]:
(imports_new > 100).value_counts()

False    23727
True       485
dtype: int64

In [11]:
(imports_new == 1).value_counts()

False    12850
True     11362
dtype: int64

There were 23,727 packages appearing in less than 100 notebooks and 11,362 packages making only one appearance. This is an indication that there are a small amount of packages that are used regularly by notebook users.

### Saving to CSV 

In [12]:
dfImports = imports_new.to_frame().reset_index()

In [13]:
dfImports.columns = ['imports', 'num_occurences'] 

In [14]:
dfImports.to_csv('../data/CSV_files/Imports.csv', index=False, encoding='utf-8')

Saving CSV file to be used in Results_200

## Heading Levels

In [15]:
df_cell_new['header_level'] = pd.DataFrame([str(line).strip('[').strip(']') for line in df_cell_new['header_level']])

In [16]:
df_headers = df_cell_new.replace('', np.nan)

In [17]:
df_headers = df_headers.dropna(subset=['header_level'])

In [18]:
df_headers['num_headings'].mean()

1.1505607194477545

There is an average of 1 heading per cell when nan values are dropped. 

In [19]:
df_headers.groupby('nb_id')['num_headings'].sum().mean()

8.280242676155389

There is an average of 8 headings per notebook.

In [20]:
df_header = df_headers['header_level'].str.split(',', expand=True).stack()

In [21]:
df_header = pd.DataFrame(df_header).reset_index()

In [22]:
df_header = df_header.drop(['level_0', 'level_1'], axis = 1) #dropping columns

In [23]:
df_header.columns = ['Heading_Level']

In [24]:
df_header['Heading_Level'] = df_header['Heading_Level'].str.strip() # cleaning empty spaces from rows

In [29]:
df_header['Heading_Level'].value_counts().sort_index(ascending=False)

'#'         187254
'##'        262790
'###'       239070
'####'       86495
'#####'      10480
'######'      2737
Name: Heading_Level, dtype: int64

In Jupyter notebooks, headers or titles are created by using the hashtag symbol (\#) in a markdown cell. The levels of the headers (ie. the size of headings and sub-headings) are denoted by the number of hashtags used. ‘\#’ level 1 header is the largest, '\#\#' level 2 is the second largest and so on. The level 3 header is the most popular followed by level 2 then by 1. This is a surprising result but can be validated. When you format notebooks, a level 1 header would be used for introductory titles and level 2 or 3 would be used for sub-headings/sub-titles which there would be more of. Altogether, there were 788,836 headers used which is a staggering amount considering there were just 200,000 notebooks. This use of headings and sub-headings in notebooks shows that many users are focusing on formatting and the presentation of their notebooks. Thus, showing us that most users are approaching their notebooks with the expectation of other people reading or interpreting it. This again is promising for the future of the literate programming style of writing and documenting code.

### Saving to CSV

In [26]:
df_header.to_csv('../data/CSV_files/Headers.csv', index=False, encoding='utf-8')

Saving CSV file to be used in Results_200